In [13]:
from model5LHzc import *
from data_1ch import *
import os, os.path

vid="v001_5" #version id should match the file number. Last number shows the cross-validation fold number
# Ran on DellWS with GeForce RTX3060 GPU

# Count the number of train and valid files
train_dir = 'mg_seg_04/train0'+vid[-1]+'/mg_seg'
train_count=len([name for name in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, name))])

valid_dir = 'mg_seg_04/valid0'+vid[-1]+'/mg_seg'
valid_count=len([name for name in os.listdir(valid_dir) if os.path.isfile(os.path.join(valid_dir, name))])

### Train with data generator

In [2]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

#Data augmentation
data_gen_args = dict(rotation_range=45,
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    shear_range=0.1,
                    zoom_range=0.1,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='wrap')

# SETTINGS ***
batch_size=3
learning_rate=1e-3

train_gen = trainGenerator(batch_size,'mg_seg_04/train0'+vid[-1],'mg_seg','mg_seg_labels',data_gen_args,save_to_dir = None)
valid_gen = trainGenerator(batch_size,'mg_seg_04/valid0'+vid[-1],'mg_seg','mg_seg_labels',data_gen_args,save_to_dir = None)

# 300 images are used for training, 52 images for validating, and 60 images for testing
train_steps = train_count//batch_size
valid_steps = valid_count//batch_size

# SETTINGS ***
loss=dice_loss
steps_per_epoch=3*train_steps
num_epochs=100

model = unet(learning_rate, loss)

opt = tf.keras.optimizers.Adam(learning_rate)
metrics = ["acc", dice_coef, iou]
model.compile(loss=loss, optimizer=opt, metrics=metrics)

callbacks = [
             ModelCheckpoint('files_xval/unet_mg_seg_'+vid+'.hdf5', verbose=1, save_best_model=True),
             ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-8),
             CSVLogger("files_xval/data_"+vid+".csv"),
             EarlyStopping(monitor="val_loss", patience=5, verbose=1)
            ]

model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=steps_per_epoch, validation_steps=valid_steps, 
                    epochs=num_epochs, callbacks=callbacks)


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Found 283 images belonging to 1 classes.
Found 283 images belonging to 1 classes.
Epoch 1/100
282/282 [==============================] - ETA: 0s - loss: 0.0858 - acc: 0.9554 - dice_coef: 0.9144 - iou: 0.8492Found 69 images belonging to 1 classes.
Found 69 images belonging to 1 classes.
282/282 [==============================] - 229s 741ms/step - loss: 0.0858 - acc: 0.9554 - dice_coef: 0.9144 - iou: 0.8492 - val_loss: 0.3325 - val_acc: 0.5113 - val_dice_coef: 0.6675 - val_iou: 0.5088

Epoch 00001: saving model to files_xval\unet_mg_seg_v001_5.hdf5
Epoch 2/100
282/282 [==============================] - 189s 670ms/step - loss: 0.0357 - acc: 0.9710 - dice_coef: 0.9642 - iou: 0.9316 - val_loss: 0.0646 - val_acc: 0.9360 - val_dice_coef: 0.9354 - val_iou: 0.8825

Epoch 00002: saving model to files_xval\unet_mg_seg_v001_5.hdf5
Epoch 3/100
282/282 [==============================] - 184s 653ms/step - loss: 0.0303 - acc: 0.9729 - dice_coef: 0.9694 - iou: 0.9415 - val_loss: 0.0359 - val_acc: 0.964

### validate your model and save predicted results

In [14]:
validGene = testGenerator("mg_seg_04/valid0"+vid[-1]+"/pred")
model.load_weights("files_xval/unet_mg_seg_"+vid+".hdf5")
results = model.predict_generator(validGene,valid_count,verbose=1)
saveResult("mg_seg_04/valid0"+vid[-1]+"/pred",results,vid[:4])

68/68 [==============================] - 8s 123ms/step


D:\research\breastCancer\mass_seg\codes\data_1ch.py:104: UserWarning: mg_seg_04/valid01/pred\62_predict_v001.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)


In [4]:
'''
trainGene = testGenerator("mg_seg_03/train03/pred")
model.load_weights("files/unet_mg_seg_"+vid+".hdf5")
results = model.predict_generator(trainGene,300,verbose=1)
saveResult("mg_seg_03/train03/pred",results,vid)
'''

'\ntrainGene = testGenerator("mg_seg_03/train03/pred")\nmodel.load_weights("files/unet_mg_seg_"+vid+".hdf5")\nresults = model.predict_generator(trainGene,300,verbose=1)\nsaveResult("mg_seg_03/train03/pred",results,vid)\n'

In [5]:
'''
testGene = testGenerator("mg_seg_03/test03/pred")
model.load_weights("files/unet_mg_seg_"+vid+".hdf5")
results = model.predict_generator(testGene,60,verbose=1)
saveResult("mg_seg_03/test03/pred",results,vid)
'''


'\ntestGene = testGenerator("mg_seg_03/test03/pred")\nmodel.load_weights("files/unet_mg_seg_"+vid+".hdf5")\nresults = model.predict_generator(testGene,60,verbose=1)\nsaveResult("mg_seg_03/test03/pred",results,vid)\n'